In [ ]:
import json
import pandas as pd
import time
import numpy as np

### Utils

In [ ]:
n_runs = 5
def average_over_runs(basename):    
    with open(basename+"_0", 'r') as file:
        result =  json.load(file)
        duration = result['configuration']['test_time']        
    p50_00_latency = [[] for _ in range(duration)]
    p99_00_latency = [[] for _ in range(duration)]
    p99_90_latency = [[] for _ in range(duration)]
    for i in range(n_runs):
        curname = basename + '_' + str(i)
        with open(curname, 'r') as file:
            result =  json.load(file)
            cur_time_series = result['ALL STATS']['Totals']['Time-Serie']
            for t in range(duration):
                p50_00_latency[t].append(cur_time_series[str(t)]["p50.00"])
                p99_00_latency[t].append(cur_time_series[str(t)]["p99.00"])
                p99_90_latency[t].append(cur_time_series[str(t)]["p99.90"])
    
    p50_00_latency_std = []
    p99_00_latency_std = []
    p99_90_latency_std = []
    for t in range(duration):
        p50_00_latency_std.append(np.std(p50_00_latency[t]))
        p99_00_latency_std.append(np.std(p99_00_latency[t]))
        p99_90_latency_std.append(np.std(p99_90_latency[t]))
        p50_00_latency[t] = np.mean(p50_00_latency[t])
        p99_00_latency[t] = np.mean(p99_00_latency[t])
        p99_90_latency[t] = np.mean(p99_90_latency[t])



    result_dict = {}
    #result_dict["sender_type"] = sender[7:]
    #result_dict["payload_size"] = size
    result_dict["n_runs"] = n_runs
    #result_dict["payload_bound"] = payload_bound
    #result_dict["circle_size"] = circle_size

    result_dict["p50.00"] = p50_00_latency
    result_dict["p99.00"] = p99_00_latency
    result_dict["p99.90"] = p99_90_latency
    
    result_dict["std_p50.00"] = p50_00_latency_std
    result_dict["std_p99.00"] = p99_00_latency_std
    result_dict["std_p99.90"] = p99_90_latency_std

    return result_dict



### Get results

In [ ]:
payload_size = [1500]
circle_size = [200]
sender_type = ['sender_rdma_write_nomultip_read', 'sender_rdma', 'sender_rdma_write_multip_write',  'sender_rdma_write_multip_read', 'sender_rdma_write_nomultip_read', 'tcpproxy']
payload_sizes = ['10-100','100-1000','1000-10000']
simult_req = ['1','10','100']
results = {}
for psize in payload_size:
    for csize in circle_size:
        for size in payload_sizes:
            for n in simult_req:
                if (size, n) not in results:
                    results[(size, n)] = {}
                # one xp, let's get th latencies and avg req rates
                #   CASE   OPS/SEC ...
                # sender | res1 res2 res3        
                data = []
                for sender in sender_type:                    
                    basename = f"results/json_{sender}_{size}_{n}_{psize}_{csize}" if sender!='tcpproxy' else f"results/json_tcpproxy_{size}_{n}"
                    result_dic = average_over_runs(basename)
                    results[(size, n)][sender] = result_dic                                                                    

In [ ]:
results

### Print results

In [ ]:
import matplotlib.pyplot as plt       

for (data_size, n) in results.keys():
    for p in ['p50.00', 'p99.00', 'p99.90']:
        cur = results[(data_size, n)]
        for sender in cur.keys():
            #plt.errorbar(range(len(cur[payload_bound][p])), cur[payload_bound]['p50.00'], yerr=cur[payload_bound][f'std_{p}'], fmt='-o', ecolor='black', capsize=5, label=f'{payload_bound}')
            plt.plot(range(len(cur[sender][p])), cur[sender][p], label=f'{sender}')
        plt.ylim(bottom=0)
        plt.xlabel('Time (in s)')
        plt.ylabel('Latency (in ms)')
        plt.title(f'{p} for workload : {data_size},{n}')
        plt.legend()        
        plt.savefig(f"plots/timeseries_{p}_{data_size}_{n}-vary-sender.svg")
        plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have time series data and std_dev lists
time_series_data = [1,2,3,4,5,6,7,8,9,10]
std_dev = [0.1,0.2,0.3,0.1,0.2,0.1,0.3,0.2,0.1,0.2]
time = range(len(time_series_data))

plt.errorbar(time, time_series_data, yerr=std_dev, fmt='-o', ecolor='black', capsize=5, label='Time Series Data')

plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()
